In [ ]:
import pathlib
import json
import itertools

# File paths

In [ ]:
dataset_path = pathlib.Path('/moby/datasets/interest_points_sts_concept')
annotations_path = dataset_path / 'json'
image_groups_path = dataset_path / 'large'
image_paths = list(itertools.chain(*[p.iterdir() for p in image_groups_path.iterdir()]))

# Detect

In [ ]:
from volumental.moby.foot_detector.detector import TFLiteDetector
from PIL import Image
import multiprocessing
import tqdm

detector_name = 'demborg_object_detection_real_images_even_longer_1583411087'
detector_file = '/moby/moby_foot_detector/experiments/{}/exported/detect.tflite'.format(detector_name)
detections_dir = annotations_path / 'detections' / detector_name
detections_dir.mkdir(exist_ok=True)

images_to_run = filter(lambda p: not detection_path_from_image_path(p).exists(), image_paths)

detector = None  # global variable to be initialized on each process
def run_detector(image_path):
    detection_path = detection_path_from_image_path(image_path)

    if not detection_path.exists():
        image = Image.open(img_path)
        result = detector.coco_detect(image, 0, normalized_coordinates=True)
        with open(detection_path, 'w') as detection_file:
            json.dump({'detections': result}, detection_file) 
            
def initialize():
    global detector
    detector = TFLiteDetector(detector_file, uniform_scaling=False, center_crop=True)
    
with multiprocessing.Pool(32, initializer=initialize) as pool:
        list(tqdm.tqdm(pool.imap_unordered(run_detector, image_paths), total=len(image_paths)))

# Crop and copy images

In [ ]:
import pprint
import pycocotools.mask as cocoUtils
import numpy as np
from PIL import Image

dataset_path = pathlib.Path('/moby/datasets/deepercut/volumental')
cropped_path = dataset_path / 'cropped'
cropped_keypoints_path = cropped_path / 'keypoints'
cropped_path.mkdir(exist_ok=True)
cropped_keypoints_path.mkdir(exist_ok=True)

for image_group_path in image_groups_path.iterdir():
    new_image_group_path = cropped_path / image_group_path.name
    new_image_group_path.mkdir(exist_ok=True)

def crop_image(image_path):
    crop_image_path = cropped_path / image_path.parent.name / image_path.name
    annotation_image_path = keypoint_path_from_cropped_image_path(image_path)
    if crop_image_path.exists() and annotation_image_path.exists():
        return
    
    annotation_path = annotation_path_from_image_path(image_path)
    detection_path = detection_path_from_image_path(image_path)
    try:
        with open(annotation_path) as annotation_file:
            annotations = json.load(annotation_file)
            image_keypoints = annotations['image_keypoints']
            bounding_box = annotations['bounding_box']
        with open(detection_path) as detection_file:
            detections = json.load(detection_file)
            detected_boxes = [d['bbox'] for d in detections['detections']]
            detected_box = select_best_detection(detected_boxes, bounding_box)
            if detected_box is None:
                #print('No box for', image_path)
                return
    except FileNotFoundError as e:
        #print('Did not find file', e)
        return

    image = Image.open(image_path)
    detected_box = np.array(detected_box)

    padding = min(detected_box[0], detected_box[1]) * 0.3
    half_padding = padding / 2.0

    detected_box[:2] = detected_box[:2] - half_padding
    detected_box[2:] = detected_box[2:] + padding

    image_size = np.array(image.size)
    detected_box = detected_box.reshape(2, 2)
    detected_box *= image_size
    detected_box = detected_box.reshape(4)
    detected_box = (*detected_box[:2], *(detected_box[:2] + detected_box[2:]))

    cropped_image = image.crop(detected_box)
    cropped_image_size = cropped_image.size

    cropped_image_keypoints = {}
    num_visible = 0
    for keypoint, (*coord, visible) in image_keypoints.items():
        cropped_image_keypoints[keypoint] = [(coord[0] * image_size[0] - detected_box[0]), 
                                             (coord[1] * image_size[1] - detected_box[1])]
        if visible:
            num_visible += 1

    if num_visible < 3:
        return

    cropped_image.save(crop_image_path)

    with open(annotation_image_path, 'w') as annotation_path:
        json.dump(cropped_image_keypoints, annotation_path)     

    #import matplotlib.pyplot as plt
    #plt.figure()
    #plt.imshow(cropped_image)
    #print(cropped_image_keypoints)
    #plt.scatter(cropped_image_keypoints['toe'][0], cropped_image_keypoints['toe'][1])
    #break
        
with multiprocessing.Pool(32) as pool:
    list(tqdm.tqdm(pool.imap_unordered(crop_image, image_paths), total=len(image_paths)))

In [ ]:
train_groups = [f'group_{i}' for i in range(7)]

cropped_train_image_paths = list(itertools.chain(*[p.iterdir() for p in cropped_path.iterdir() if p.name in train_groups]))

In [ ]:
with open(next(cropped_keypoints_path.iterdir())) as keypoints_file:
    keypoints = json.load(keypoints_file)

keypoint_to_id = {keypoint: i for i, keypoint in enumerate(sorted(keypoints.keys()))}

In [ ]:
from PIL import Image
import numpy as np

dataset = []
for image_path in tqdm.tqdm(cropped_train_image_paths):
    keypoint_path = keypoint_path_from_cropped_image_path(image_path)
    try:
        with open(keypoint_path) as keypoint_file:
            keypoints = json.load(keypoint_file)
    except:
        continue
    keypoints = [[keypoint_to_id[keypoint], coords[0], coords[1]] for keypoint, coords in keypoints.items()]
    entry = {
        'image': str(image_path),
        'size': [3, *image_size],
        'joints': keypoints
    }
    #entry = [str(image_path), [3, *image_size], joints_dict]
    dataset.append(entry)
    #if i > 10:
    #    break
        

In [ ]:
import scipy.io as sio
sio.savemat('dataset_python_train.mat', {'dataset': dataset})

In [ ]:
dataset_loaded = sio.loadmat('dataset.mat')['dataset']